In [0]:
from argparse import ArgumentParser
import logging

from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST

In [3]:
pip install pytorch-ignite

     |████████████████████████████████| 92kB 6.1MB/s 


In [0]:
from ignite.engine import (
    Events, create_supervised_trainer, create_supervised_evaluator)
from ignite.metrics import Accuracy, Loss
from ignite.handlers import ModelCheckpoint

In [0]:
## 신경망 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=-1)

In [0]:
## MNIST data 로드, 정규화
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

    train_loader = DataLoader(
        MNIST(
            download=True, root=".", transform=data_transform, train=True),
        batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(
        MNIST(
            download=False, root=".", transform=data_transform, train=False),
        batch_size=val_batch_size, shuffle=False)
    return train_loader, val_loader

In [0]:
def run(train_batch_size, val_batch_size,
        epochs, lr, momentum,
        log_interval, restore_from, crash_iteration=1000):

    train_loader, val_loader = get_data_loaders(
        train_batch_size, val_batch_size)
    model = Net()
    device = 'cpu'
    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    trainer = create_supervised_trainer(
        model, optimizer, F.nll_loss, device=device)
    evaluator = create_supervised_evaluator(model,
                                            metrics={'accuracy': Accuracy(),
                                                     'nll': Loss(F.nll_loss)},
                                            device=device)
    # Setup debug level of engine logger:
    trainer._logger.setLevel(logging.INFO)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = logging.Formatter(
        "%(asctime)s|%(name)s|%(levelname)s| %(message)s")
    ch.setFormatter(formatter)
    trainer._logger.addHandler(ch)

    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        iter = (engine.state.iteration - 1) % len(train_loader) + 1
        if iter % log_interval == 0:
            print("Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
                  "".format(
                      engine.state.epoch, iter,
                      len(train_loader), engine.state.output))

        if engine.state.iteration == crash_iteration:
            raise Exception("STOP at {}".format(engine.state.iteration))

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['nll']
        print(
            "Training Results - Epoch: {}\
            Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
                engine.state.epoch, avg_accuracy, avg_nll))

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll = metrics['nll']
        print(
            "Validation Results - Epoch: {}\
            Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
                engine.state.epoch, avg_accuracy, avg_nll))

    objects_to_checkpoint = {"model": model, "optimizer": optimizer}
    engine_checkpoint = ModelCheckpoint(
        dirname="engine_checkpoint",
        filename_prefix='ignite_checking',
        require_empty=False,
        save_interval=100)
    trainer.add_event_handler(
        Events.ITERATION_COMPLETED, engine_checkpoint, objects_to_checkpoint)

    if restore_from == "":
        trainer.run(train_loader, max_epochs=epochs)
    else:
        raise NotImplementedError('Not implemented yet')

In [8]:
if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument('--batch_size', type=int, default=64,
                        help='input batch size for training (default: 64)')
    parser.add_argument('--val_batch_size', type=int, default=1000,
                        help='input batch size for validation (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10,
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01,
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5,
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--log_interval', type=int, default=300,
                        help='how many batches to wait before logging')
    parser.add_argument('--restore_from', type=str, default="",
                        help='location from where the model can be reloaded')
    parser.add_argument(
        '--crash_iteration', type=int, default=1000,
        help='Iteration to suddenly raise as exception')
    args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]
                             [--val_batch_size VAL_BATCH_SIZE]
                             [--epochs EPOCHS] [--lr LR] [--momentum MOMENTUM]
                             [--log_interval LOG_INTERVAL]
                             [--restore_from RESTORE_FROM]
                             [--crash_iteration CRASH_ITERATION]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-6a721a57-45f7-4fce-a50c-3928f545314a.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
